In [16]:
import numpy as np
import pandas as pd
import do_mpc as mpc
from casadi import *
import sys

In [ ]:
def model():
    # Create a do-mpc model
    m = mpc.model.Model('continuous')

    # Define system variables
    servo_pos = m.set_variable(var_type='_x', var_name='servo_pos', shape=(1,1))
    servo_vel = m.set_variable(var_type='_x', var_name='servo_vel', shape=(1,1))

    target_pos = m.set_variable(var_type='_u', var_name='target_pos', shape=(1,1))
    target_vel = m.set_variable(var_type='_u', var_name='target_vel', shape=(1,1))

    # Define system equations
    dservo_pos = m.set_rhs('servo_pos', servo_vel)
    # Assume the control input directly sets the velocity
    dservo_vel = m.set_rhs('servo_vel', m.set_variable(var_type='_u', var_name='control_vel', shape=(1,1)))

    m.set_rhs('servo_pos', dservo_pos)
    m.set_rhs('servo_vel', dservo_vel)

    m.setup()
    return m

def controller(model):
    # Setup MPC controller
    mpc = mpc.controller.MPC(model)

    setup_mpc = {
        'n_horizon': 10,
        't_step': 0.1,
        'store_full_solution': True,
    }
    mpc.set_param(**setup_mpc)

    # Weights for the cost function
    w_pos = 1.0  # Weight for position error
    w_vel = 1.0  # Weight for velocity error

    # Define objective function
    pos_error = model.aux['servo_pos'] - model.aux['target_pos']
    vel_error = model.aux['servo_vel'] - model.aux['target_vel']

    mterm = w_pos * pos_error**2 + w_vel * vel_error**2  # Terminal cost
    lterm = w_pos * pos_error**2 + w_vel * vel_error**2  # Stage cost

    mpc.set_objective(mterm=mterm, lterm=lterm)
    mpc.set_rterm(control_vel=1)  # Penalty on control velocity

    # Define velocity constraints
    max_servo_velocity =  # Define the maximum servo velocity
    mpc.bounds['lower','_u', 'control_vel'] = -max_servo_velocity
    mpc.bounds['upper','_u', 'control_vel'] = max_servo_velocity

    mpc.setup()
    return mpc

# Initialize model and controller
model = model()
mpc = controller(model)

# Simulation setup (for testing)
simulator = mpc.simulator.Simulator(model)
simulator.set_param(t_step = 0.1)
simulator.setup()

# Initial state
x0 = np.array([[0], [0]])
simulator.x0 = x0
mpc.x0 = x0

# Target position and velocity (example values)
target_position = 1.0
target_velocity = 0.5

# Simulation loop
for k in range(20):
    u0 = np.array([[target_position], [target_velocity]])
    y_next = simulator.make_step(u0)
    mpc.make_step(y_next)


In [18]:
# Given tangential velocity and radius, calculate angular velocity in degrees per second

tangential_velocity = 15  # m/s
radius = 2  # meters

# Angular velocity in radians per second
angular_velocity_rad = tangential_velocity / radius

# Convert to degrees per second
angular_velocity_deg = angular_velocity_rad * (180 / np.pi)  # radians to degrees conversion
angular_velocity_deg

429.7183463481174